### Fine-tuning 6-Billion GPT-J in colab with LoRA and 8-bit compression

This notebook is a proof of concept for fine-tuning [GPT-J-6B](https://huggingface.co/EleutherAI/gpt-j-6B) with limited memory. A detailed explanation of how it works can be found in [this model card](https://huggingface.co/hivemind/gpt-j-6B-8bit).

In [1]:
import transformers

import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise

from tqdm.auto import tqdm

/home/yangj13/miniconda3/envs/GPTJ/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==============================WARNING: DEPRECATED!==============================
WARNING! This version of bitsandbytes is deprecated. Please switch to `pip install bitsandbytes` and the new repo: https://github.com/TimDettmers/bitsandbytes
==============================WARNING: DEPRECATED!==============================


### Converting the model to 8 bits.

We convert EleutherAI's GPT-J-6B model to 8 bits using facebook's [bitsandbytes](https://github.com/facebookresearch/bitsandbytes) library. This reduces the model's size from 20Gb down to just 6Gb.

Note that we don't convert linear layer biases to 8 bit as they take up less that 1% of the model's weight anyway.

In [2]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias

    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output

    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"


class DequantizeAndLinear(torch.autograd.Function):
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias).clone()

    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias


class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None

    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output

    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"


def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)

    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)


def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [3]:
# class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
#     def __init__(self, config):
#         super().__init__(config)

#         convert_to_int8(self.attn)
#         convert_to_int8(self.mlp)


# class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
#     def __init__(self, config):
#         super().__init__(config)
#         convert_to_int8(self)


class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


# transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

In [5]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [6]:
gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

lm_head Linear(in_features=4096, out_features=50400, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, b

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

### Text generation example

In [50]:
device = 'cuda'
gpt.to(device)
prompt = tokenizer("The results of a systematic study of the otological aspects in 13 cases of earpit-deafness syndrome are reported. The audiometric, radiological and vestibular findings as well as the results of exploratory tympanotomies with and without stapedectomies are discussed together with the results reported in the literature. A convincing explanation of the poor results of exploratory tympanotomies in cases with mixed hearing loss is not furnished. If the hearing loss is confined to conduction and ankylosis of the stapes or a disconnection of the ossicular chain is suspected, exploratory tympanotomy can be expected to be successful. \n\n##\n\n", return_tensors='pt')
prompt = {key: value.to(device) for key, value in prompt.items()}
out = gpt.generate(**prompt, min_length=128, max_length=512, do_sample=True)
tokenizer.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"The results of a systematic study of the otological aspects in 13 cases of earpit-deafness syndrome are reported. The audiometric, radiological and vestibular findings as well as the results of exploratory tympanotomies with and without stapedectomies are discussed together with the results reported in the literature. A convincing explanation of the poor results of exploratory tympanotomies in cases with mixed hearing loss is not furnished. If the hearing loss is confined to conduction and ankylosis of the stapes or a disconnection of the ossicular chain is suspected, exploratory tympanotomy can be expected to be successful. \n\n##\n\n  TABLES\n\nPURPOSE OF THE STUDY: To demonstrate the usefulness of exploratory tympanotomy in the assessment of cases of ear pathology with hearing loss. METHODS/PROCEDURE: This retrospective study involves the examination of patients treated for ear pathology during the period from August 1994 to September 2000 at the Otolaryngology Department, ENT Clin

earpit | HP_0004467\n deafness | HP_0000404\n mixed hearing loss | HP_0000410\n hearing loss | HP_0000365\n conduction | HP_0000405\n ankylosis of the stapes | HP_0000381\n disconnection of the ossicular chain | HP_0004452\n END'

### LoRA fine-tuning example
Here we demonstrate how to fine-tune the proposed model using low-rank adapters [(Hu et al, 2021)](https://arxiv.org/abs/2106.09685) and [8-bit Adam](https://arxiv.org/abs/2110.02861). We also use [dataset streaming API](https://huggingface.co/docs/datasets/dataset_streaming.html) to avoid downloading the large dataset.

In [4]:
def add_adapters(model, adapter_dim=16):
    assert adapter_dim > 0

    for module in model.modules():
        if isinstance(module, FrozenBNBLinear):
            module.adapter = nn.Sequential(
                nn.Linear(module.in_features, adapter_dim, bias=False),
                nn.Linear(adapter_dim, module.out_features, bias=False),
            )
            nn.init.zeros_(module.adapter[1].weight)
        elif isinstance(module, FrozenBNBEmbedding):
            module.adapter = nn.Sequential(
                nn.Embedding(module.num_embeddings, adapter_dim),
                nn.Linear(adapter_dim, module.embedding_dim, bias=False),
            )
            nn.init.zeros_(module.adapter[1].weight)

# add_adapters(gpt)
# gpt.to(device)

In [8]:
from datasets import load_dataset
from bitsandbytes.optim import Adam8bit

gpt.gradient_checkpointing_enable()

# codeparrot = load_dataset("transformersbook/codeparrot-train", streaming=True)
optimizer = Adam8bit(gpt.parameters(), lr=1e-5)

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
import pandas as pd
# path = '/content/drive/MyDrive/biolarkgsc_locs.csv'
path = './biolarkgsc_locs.csv'
biolark = pd.read_csv(path, sep='\t')

In [8]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(biolark, test_size=0.1, random_state=42)
len(train_df), len(test_df)

(205, 23)

In [9]:
def preprocess(train_df):
    rows = []
    for i, row in train_df.iterrows():
#         bad examples in train_df
#         if i in {160,7,23,76}:
#             continue
        list_hpo = row.labels.split(';')
        list_hpo_pair = [[_.split('|')[0], _.split('|')[1].split(':')]  for _ in list_hpo]
        # print(list_hpo_pair)
        # for hpo, [s,e] in list_hpo_pair:
        #     print(row.text[int(s)-50:int(e)+50])
        # break
        ans = ""
        hpos = set()
        for hpo, [s,e] in list_hpo_pair:
            if hpo in hpos:
                continue
            ans += ' ' + row.text[int(s):int(e)] + ' | ' + hpo + '\n'
            hpos.add(hpo)
        ans += ' END'
        text = row.text+' \n\n##\n\n '+ans
        rows.append(text)
#         rows.append({"prompt":f"{row.text}\n\n###\n\n", "completion":f" {ans}"})
    return rows

In [10]:
biolark_ft = preprocess(train_df)
biolark_ft

['Familial Angelman syndrome (AS) can result from mutations in chromosome 15q11q13 that, when transmitted from father to child, result in no phenotypic abnormality but, when transmitted from mother to child, cause AS. These mutations therefore behave neither as dominant nor as recessive mutations but, rather, show an imprinted mode of inheritance. We have analyzed two sibling pairs with AS and a larger family with four AS offspring of three sisters with several recently described microsatellite polymorphisms in the AS region. AS siblings inherited the same maternal alleles at the GABRB3 and GABRA5 loci, and the unaffected siblings of AS individuals inherited the other maternal alleles at these loci. In one of the AS sibling pairs, analysis of a recombination event indicates that the mutation responsible for AS is distal to locus D15S63. This result is consistent with a previously described imprinted submicroscopic deletion causing AS, a deletion that includes loci D15S10, D15S113, and 

In [11]:
biolark_tt = preprocess(test_df)

In [16]:
#fine-tune biolark
for epoch in range(5):
    print(epoch)
    for i in tqdm(range(len(biolark_ft))):
        if len(biolark_ft[i]) <= 1:
            continue

        batch = tokenizer(biolark_ft[i], truncation=True, max_length=2048, return_tensors='pt')
        batch = {k: v.cuda() for k, v in batch.items()}
        with torch.cuda.amp.autocast():
            out = gpt.forward(**batch,)

            loss = F.cross_entropy(out.logits[:, :-1, :].flatten(0, -2), batch['input_ids'][:, 1:].flatten(),
                                   reduction='mean')
        print(loss)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

0


  0%|          | 0/205 [00:00<?, ?it/s]

tensor(0.0170, device='cuda:0', grad_fn=<NllLossBackward0>)


  1%|          | 2/205 [00:00<01:09,  2.93it/s]

tensor(0.0440, device='cuda:0', grad_fn=<NllLossBackward0>)


...

tensor(0.1696, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|█████████▉| 204/205 [00:57<00:00,  3.66it/s]

tensor(0.2714, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 205/205 [00:57<00:00,  3.55it/s]

tensor(0.0633, device='cuda:0', grad_fn=<NllLossBackward0>)


In [43]:
torch.save(gpt.state_dict(), './torch_model/model.pth')
gpt.save_pretrained('./model/')

# prediction

In [7]:
def add_adapters(model, adapter_dim=16):
    assert adapter_dim > 0

    for module in model.modules():
        if isinstance(module, FrozenBNBLinear):
            module.adapter = nn.Sequential(
                nn.Linear(module.in_features, adapter_dim, bias=False),
                nn.Linear(adapter_dim, module.out_features, bias=False),
            )
            nn.init.zeros_(module.adapter[1].weight)
        elif isinstance(module, FrozenBNBEmbedding):
            module.adapter = nn.Sequential(
                nn.Embedding(module.num_embeddings, adapter_dim),
                nn.Linear(adapter_dim, module.embedding_dim, bias=False),
            )
            nn.init.zeros_(module.adapter[1].weight)

# add_adapters(gpt)
# gpt.to(device)

In [8]:
gptmodel = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)


lm_head Linear(in_features=4096, out_features=50400, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, b

In [9]:
add_adapters(gptmodel)

In [10]:
gptmodel.load_state_dict(torch.load("./torch_model/model.pth"))

<All keys matched successfully>

In [9]:
prompt = "We describe the clinical findings of 15 individuals in a large kindred affected with distal arthrogryposis type 1A (DA1A). The most consistent findings among individuals were overlapping fingers at birth, abnormal digital flexion creases, and foot deformities, including talipes equinovarus and vertical talus. There was marked intrafamilial variation in the expression of DA1A. Linkage mapping of the locus for DA1A suggests that the use of strict diagnostic criteria excludes unaffected individuals rigorously, but can produce incomplete ascertainment of affected individuals. In the context of an affected family, the range of phenotypes consistent with a diagnosis of DA1A needs to be expanded.\n\n##\n\n"

In [34]:
prompt = "A 5 years old girl with swallowing difficulty, recurrent chest infections and developmental delay. She can only say 4 words. Her parents are first-degree cousins, and there is no family history of a similar condition. Her growth parameters at 5 years of age were weight 9.4 kg, height 82.5 cm and head circumference 46 cm, all below the 3rd percentile. Her physical examination was significant for a blue sclera, hypotonia and hyporeflexia. Vision and hearing were normal, and other examinations were within normal limits. The brain MRI was grossly normal with diffuse T-2 hyperintensity in subcortical white matter. \n\n##\n\n"

In [35]:
prompt = tokenizer(prompt, return_tensors='pt')

In [37]:
device_1 = 'cuda'
gptmodel.to(device_1)
# prompt = tokenizer("The results of a systematic study of the otological aspects in 13 cases of earpit-deafness syndrome are reported. The audiometric, radiological and vestibular findings as well as the results of exploratory tympanotomies with and without stapedectomies are discussed together with the results reported in the literature. A convincing explanation of the poor results of exploratory tympanotomies in cases with mixed hearing loss is not furnished. If the hearing loss is confined to conduction and ankylosis of the stapes or a disconnection of the ossicular chain is suspected, exploratory tympanotomy can be expected to be successful. \n\n##\n\n", return_tensors='pt')
prompt = {key: value.to(device_1) for key, value in prompt.items()}
out = gptmodel.generate(**prompt, min_length=128, max_length=600, 
                        temperature=0.1,
                        top_p=0.5,
                        do_sample=True)
tokenizer.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'A 5 years old girl with swallowing difficulty, recurrent chest infections and developmental delay. She can only say 4 words. Her parents are first-degree cousins, and there is no family history of a similar condition. Her growth parameters at 5 years of age were weight 9.4 kg, height 82.5 cm and head circumference 46 cm, all below the 3rd percentile. Her physical examination was significant for a blue sclera, hypotonia and hyporeflexia. Vision and hearing were normal, and other examinations were within normal limits. The brain MRI was grossly normal with diffuse T-2 hyperintensity in subcortical white matter. \n\n##\n\n  swallowing difficulty | HP_0001627\n developmental delay | HP_0001263\n hypotonia | HP_0001159\n hyporeflexia | HP_0008586\n 3 rd percentile | HP_0004051\n low weight, 9.4 k | HP_0006996\n low height, 82.5 cm | HP_0009373\n low head circumference, 46 cm | HP_0004458\n hypoplastic scul | HP_0001140\n hypoplastic scul | HP_0004452\n brachycephaly | HP_0001155\n brachyce

In [18]:
device_1 = 'cuda'
gptmodel.to(device_1)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

In [16]:
def gpt_val(prompt):
    prompt = tokenizer(prompt, return_tensors='pt')
    prompt = {key: value.to(device_1) for key, value in prompt.items()}
    out = gptmodel.generate(**prompt, min_length=128, max_length=600, 
                        temperature=0.1,
                        top_p=0.5,
                        do_sample=True)
    print(tokenizer.decode(out[0]))

In [30]:
for id, prompt in enumerate(biolark_tt):
    
    prompt = prompt.split('\n\n##\n\n')[0] + '\n\n##\n\n'
    print(f'------{id}------')
    gpt_val(prompt)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


------0------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Nevoid basal cell carcinoma syndrome (NBCCS) is rare in black persons. We describe an 11-year-old black boy with NBCCS who presented with exotropia and a painful, expanding, cystic mass in the left posterior alveolar ridge. Further examination revealed odontogenic keratocysts with palmar and plantar pitting. Less than 5% of reported patients with NBCCS are black. To our knowledge, this is the first report of a black patient with NBCCS presenting with exotropia and an impacted molar displaced into the orbit by an odontogenic keratocyst. 

##

  exotropia | HP_0001144
 rare disorders | HP_0003745
 odontogenic keratocysts | HP_0010603
 palmar and plantar pitting | HP_0006860
 molar | HP_0009794
 cystic mass | HP_0007678
 expansion | HP_0003813
 END_REPORT | HP_0004467
 END_RECOMMEND | HP_0004411
 exotropia | HP_0001193
 odontogenic keratocysts | HP_0010603
 palmar and plantar pitting | HP_0007678
 plantar pitting | HP_0006812
 pitting | HP_0006860
 molar | HP_0009794
 cystic mass | HP_000

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Neurofibromatosis type 2 (NF2) is a monogenic dominantly inherited disease predisposing carriers to develop nervous system tumours. To identify the genetic defect, the region between two flanking polymorphic markers on chromosome 22 was cloned and several genes identified. One is the site of germ-line mutations in NF2 patients and of somatic mutations in NF2-related tumours. Its deduced product has homology with proteins at the plasma membrane and cytoskeleton interface, a previously unknown site of action of tumour suppressor genes in humans. 

##

  Neurofibromatosis | HP_0006746
 dominantly inherited disease | HP_0000006
 tumour | HP_0002664
 somatic mutations | HP_0001425
 END_CLASS959000 | HP_0007121
 END_CLASS65000 | HP_0009794
 END_CLASS68000 | HP_0009795
 END_CLASS16000 | HP_0004467
 plasma membrane | HP_0001059
 cytoskeleton | HP_0003121
 somatic mutations in NF2-related tumours | HP_0009792
 NF2 patients | HP_0009588
 relevant loci | HP_0009911
 relevant loci, between two fla

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bilateral acoustic neurofibromatosis (BANF) is a severe autosomal dominant disorder involving development of multiple tumours of the nervous system including meningiomas, gliomas, neurofibromas and particularly bilateral acoustic neuromas. We have used genetic linkage analysis with DNA markers to establish that the defective gene causing BANF is on chromosome 22, and is therefore distinct from the gene for the von Recklinghausen form of neurofibromatosis, which maps to chromosome 17. Linked DNA markers will be particularly valuable in BANF, facilitating early detection of tumours and thereby permitting more effective surgical intervention. In view of the reported loss of genes on chromosome 22 in meningiomas and acoustic neuromas, the genetic localization of the primary BANF defect strongly supports the concept that the disease locus encodes a 'tumour suppressor' gene. Isolation of this gene should provide insights into the pathogenesis of acoustic neuromas and other nervous system tum

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Three successive generations in two families affected with the popliteal pterygium syndrome are reported. While expression of the syndrome was relatively mild in the first and second generation, the patients in the third generation showed the full-blown syndrome. Differential diagnosis between mildly affected patients with the popliteal pterygium syndrome and those with Van der Woude syndrome is difficult and may even be impossible. The present observations further support the hypothesis that both syndromes may in fact represent variants of the same condition. 

##

  popliteal pterygium | HP_0009756
 pterygium | HP_0001059
 popliteal pterygium syndrome | HP_0009757
 mildly affected | HP_0004467
 Van der Woude syndrome | HP_0100257
 differential diagnosis | HP_0004713
 END_CLASSROOM(HP_0009756) | CPG_241500
 END_CLASSROOM(HP_0001059) | CPG_3US | CPG_000300
 END_CLASSROOM(HP_0001508) | HMW_HORVATIC_CURVATURAL | HP_0004707
 END_CLASSROOM(HP_0004467) | HMW_HORVATIC_CURVATURAL | HP_0004707

This training loop is just a proof of concept - to show that even in the heaviest case, it still fits on a gpu.
Depending on your finetuning task, you'll need to remove some parts.
Below we explain how to modify the code to achieve the setup from the [LoRA paper](https://arxiv.org/pdf/2106.09685.pdf)

If you wanna fine-tune a-la LoRA , please use the parameters from Table 11,12 and 15 as a starter:

(1) Train only the adapter matrices from attention layers

In the above example, we train all kinds of adapters, and also layernorm scales and biases. This is only useful for fine-tuning over reasonably large datasets over long time.
For quick setups you should tag everything except **the attention adapters** as `requires_grad=False` -- or just don't feed them into Adam:

```

params_for_optimizer = [
    param for name, param in model.named_parameters()
    if "attn" in name and "adapter" in name
]
print("Trainiable params:", len(params_for_optimizer))

# and after you verified it:
for name, param in model.named_parameters():
    if param not in params_for_optimizer:
        print(f"Setting {name} requires_grad=False")
        param.requires_grad = False
```

An even better way is to only create adapters that you need by modifying the `add_adapters` function above:
```
for name, module in model.named_modules():
    if isinstance(module, (FrozenBNBLinear, FrozenBNBEmbedding)):
        if "attn" in name:
            print("Adding adapter to", name)

            todo_initialize_adapters_like_in_notebook()
        else:
            print("Not adding adapter to", name)
```
As a side-effect, that would actually somewhat reduce the memory usage and may let you fit a longer sequence (e.g. 256)


(2) initialize the second adapter matrix with zeros
```
for name, module in model.named_modules():
    if hasattr(module, "adapter"):
        print("Initializing", name)
        nn.init.zeros_(module.adapter[1].weight)
        # optional: scale adapter[0].weight by (LoRA_alpha / r)
```

(3) use warmup and weight decay in Adam:
```
optimizer = Adam8Bit(..., weight_decay=0.01)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps_from_paper(), expected_total_number_of_steps
)

actually_use_scheduler_in_training_loop()
```

Finally, we recommend modifying training loop to track the training metrics, saving the best checkpoint, etc.

In [11]:
#run ID-68
import os

notes = os.listdir('/mnt/isilon/wang_lab/jingye/projects/data/ID-68/corpus/')
notes

['17DG0781',
 '09DG00835',
 '16DG0402',
 '13DG0911',
 '14DG1188',
 '17DG0766',
 '17DG0769',
 '17DG0773',
 '12DG1370',
 '14DG0647',
 '15DG0298',
 '15DG0299',
 '17DG0783',
 '17DG0768',
 '15DG0315',
 '16DG1608',
 '16DG0485',
 '17DG0777',
 '15DG0307',
 '17DG0764',
 '17DG0776',
 '17DG0774',
 '17DG0775',
 '16DG1123',
 '16DG0971',
 '15DG2492',
 '15DG2032',
 'DD_91704',
 '15DG0837',
 '15DG0633',
 '12DG2311',
 '15DG2336',
 '17DG0771',
 '15DG0305',
 '13DG1545',
 '13DG1665',
 '16DG1625',
 '11DG1842',
 '16DG0201',
 '10DG0720',
 '14DG2158',
 '16DG0105',
 '17DG0780',
 '15DG0115',
 '15DG2064',
 '15DG0749',
 '14DG0959',
 '17DG0778',
 '13DG1202',
 '17DG0767',
 '16DG0325',
 '17DG0779',
 '17DG0763',
 '14DG1743',
 '10DG0840',
 '17DG0772',
 '17DG0782',
 '14DG0467',
 '15DG2661',
 '17DG0762',
 '17DG0770',
 '16DG0697',
 '16DG0051',
 '15DG2307',
 '15DG2206',
 '14DG1320',
 '14DG0056',
 '17DG0765']

In [12]:
id_68 = []
id_68_lb = [] 
for note in notes:
    with open(f'/mnt/isilon/wang_lab/jingye/projects/data/ID-68/corpus/{note}','r') as file:
        id_68.append(file.read())
    with open(f'/mnt/isilon/wang_lab/jingye/projects/data/ID-68/ann/{note}', 'r') as file:
        id_68_lb.append(file.read())

In [13]:
a = list(zip(id_68, id_68_lb))
import random
random.seed(1)
a = list(zip(id_68,id_68_lb))

In [27]:
my_list = []
for prompt, lb in zip(id_68, id_68_lb):
    if len(lb.rstrip('\n').split('\n')) <= 10:
        my_list.append((prompt,lb))

In [28]:
len(my_list)

31

In [29]:
random.seed(1)
new_list = random.sample(my_list, 20)

In [32]:
i = 1
for prompt, lb in new_list:
    print(f'---{i}---')
    print(prompt)
    print(lb)
    print(len(lb.rstrip('\n').split('\n')))
    i += 1

---1---
This is a 13 months old boy with severe microcephaly, Dandy-Walker malformation, spasticity, GDD, developmental regression and failure to thrive. Parents are consanguineous, and there are no affected siblings. Head circumference at birth was 27 cm and at 1 year was 34 cm.
40	52	microcephaly	HP:0000252
54	79	dandy walker malformation	HP:0001305
81	91	spasticity	HP:0001257
98	122	developmental regression	HP:0002376
127	144	failure to thrive	HP:0001508

5
---2---
A 9 months old deceased boy who was born at full-term via C-section due to meconium-stained liquor and fetal distress. He was admitted to NICU immediately after birth where he developed convulsions and jerky movements. His seizures have been difficult to control, and he is on multiple medications. The parents are first-degree cousins once removed, and they had a daughter with a similar course that passed away at the age of 16 months. His examination was significant for dysmorphic features and sluggish pupils. Brain MRI sh

In [33]:
i = 1
for prompt, lb in new_list:
    prompt = prompt + '\n\n##\n\n'
    print(f'------{i}------')
    gpt_val(prompt)
    i += 1

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


------1------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


This is a 13 months old boy with severe microcephaly, Dandy-Walker malformation, spasticity, GDD, developmental regression and failure to thrive. Parents are consanguineous, and there are no affected siblings. Head circumference at birth was 27 cm and at 1 year was 34 cm.

##

  severe microcephaly | HP_0008635
 microcephaly | HP_0001156
 Dandy-Walker malformation | HP_0009589
 spasticity | HP_0001250
 GDD | HP_0001263
 developmental regression | HP_0007087
 failure to thrive | HP_0002089
 Consangineous | HP_0003745
 END_CLASS | HP_0004322
 END_CLASS | HP_0001425
 13 months old boy | HP_0003593
 growth retardation | HP_0001263
 END_CLASS | HP_0004467
 ILL | HP_0003812
  spasticity, GDD | HP_0002564
  spasticity | HP_0001250
  microcephaly | HP_0001999
  Consangineous | HP_0003745
  END_CLASS | HP_0004412
  END_CLASS | HP_0003812
  HP_0003812
  GDD | HP_0001263
  END_CLASS | HP_0004712
  HP_0004745
  developmental regression | HP_0007087
  failure to thrive | HP_0002089
  growth retarda

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

A 3 years old girl with failure to thrive, short stature, microcephaly, speech delay, anteriorly placed anus and dysmorphic features. She was born preterm with IUGR to consanguineous parents. 

##

  failure to thrive | HP_0001061
 short stature | HP_0004375
 microcephaly | HP_0000252
 speech delay | HP_0001177
 anteriorly placed anus | HP_0002089
 dysmorphic features | HP_0001249
 preterm | HP_0002814
 IUGR | HP_0001425
 consanguineous parents | HP_0006746
 END_CLASSIFICATION
 END_CLASSIFICATION3RICH_BASIC_SYNDROMS | HP_0006322
 preterm | HP_0001425
 IUGR | HP_0001425
 END_CLASSIFICATION2CUTANEOUS_SYMPTOMS | HP_0004711
 dysmorphic features | HP_0001250
 END_CLASSIFICATION1CUTANEOUS_SYMPTOMS | HP_0001263
 END_CLASSIFICATION1CUTANEOUS | HP_0004711
 END_CLASSIFICATION2VAGINAL_CRY | HP_0005134
 END_CLASSIFICATION3RICH_BASIC | HP_0009795
 END_CLASSIFICATION1IUGR | HP_0001425
 END_CLASSIFICATION2CUTANEOUS_SYMPTOMS_VAGINAL_CRY | HP_0005136
 IUGR | HP_0001425
 END_CLASSIFICATION3RICH_BASIC_SY

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A 5 years old boy with dysmorphic facial features, hypertelorism, downslanting eyes, small ears, epicanthal folds, short neck, global developmental delay and mild autistic features. The parents are consanguineous, and they have another affected daughter.

##

  dysmorphic facial features | HP_0004467
 hypertelorism | HP_0000209
 downslanting eyes | HP_0000356
 small ears | HP_0000394
 epicanthal folds | HP_0000381
 short neck | HP_0004412
 global developmental delay | HP_0001263
 mild autistic features | HP_0001263
 END_OF_PROJECTION | HP_0004422
 Consanguineous | HP_0001425
 global developmental delay | HP_0001263
 mild autistic features | HP_0001263
 END_OF_PROJECT | HP_0003813
 END_OF_HEAD | HP_0004450
 developmental delay | HP_0001263
 Autistic features | HP_0001263
 END_OF_HEAD | HP_0004449
 Autistic features | HP_0001263
 END_OF_BEHAVIOR | HP_0004429
 Consanguineous | HP_0001425
 END_OF_BEHAVIOR | HP_0004428
 Autistic features | HP_0001263
 END_OF_BEHAVIOR | HP_0004427
 Autistic 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A 17 years old girl with neuroregression that started at 4 years of age. She had been developmentally normal until the age of 4 years when she started to gradually lose her ability to walk and talk, and she is currently wheelchair-bound. She communicates through nodding her head. She also has developed seizures, but they have been under control. The parents are first-degree cousins, and they have three other affected children and three deceased children who died for an unknown reason. On examination, her growth parameters were weight 32 kg (<3rd percentile) and head circumference 49 cm(<3rd percentile). She has an ulnar deviation of the hands. She also has hyperreflexia of the extremities and strabismus. MRI showed mild leukodystrophy. Skin biopsy showed inclusion bodies. 

##

  neuroregression | HP_0001311
 developmentally normal | HP_0004322
 gradually lost her ability to walk and talk | HP_000ID35
 loss of her ability to walk and talk | HP_0004711
 gradually lost her ability to wal